In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 14.4 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import csv
import time

In [ ]:
model = YOLO('yolov9c.pt')

cap = cv2.VideoCapture('/content/drive/MyDrive/اليوم الثالث -ليل.mp4')

# Get width, height, and fps
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)


print("Width:", w)
print("Height:", h)
print("FPS:", fps)

ret, frame = cap.read()



Width: 480
Height: 848
FPS: 29.99175824175824


In [ ]:

points = np.array([
    (100, h//8),
    (5, h),
    (w, h),
    (w, h//8),
    (w-900, h//8),
    (400, h//8)
])
cv2.polylines(frame, [points], isClosed=True, color=(255, 0, 0), thickness=2)


from google.colab.patches import cv2_imshow
cv2_imshow(frame)

In [ ]:
output_path = 'Day3_at_night.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

In [ ]:
vehicle_classes = [2, 3, 5, 7]  # YOLOv8 IDs for cars, motorcycle, busses, trucks

class_name = ['heavy','mid','light']

In [ ]:
def count_vehicles(detections, vehicle_classes):
    counts = {cls: 0 for cls in vehicle_classes}
    for detection in detections:
        class_id = int(detection.cls.item())
        if class_id in vehicle_classes:
            counts[class_id] += 1
    return counts

In [ ]:
def is_point_in_polygon(point, vertices):
    result = cv2.pointPolygonTest(vertices, point, False)
    return result >= 0

In [ ]:
# to get current timestamp
timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())

while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    detection_frame = frame.copy()

    results = model.predict(detection_frame)
    processed_frame = results[0].plot(line_width=1)

    cv2.polylines(processed_frame, [points], isClosed=True, color=(240, 207, 137), thickness=2)

    count = 0

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        centroid = ((x1 + x2) // 2, (y1 + y2) // 2)

        if is_point_in_polygon(centroid, points):
            count += 1

    text = ''
    background_color = ()


    if count > 20:
          with open('detections_Day3_at_Day.csv', 'a', newline='') as file:
               writer = csv.writer(file)
               writer.writerow([class_name[0], timestamp, count, 'King Fahad Road'])

          text = f"Number of Vehicles: {count} | Traffic: {class_name[0]}"
          background_color = (0, 0, 128)

    elif count in range (10,20):
          with open('detections.csv', 'a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([class_name[1], timestamp, count, 'King Fahad Road'])

          text = f"Number of Vehicles: {count} | Traffic: {class_name[1]}"
          background_color = (52, 191, 255)

    else:
          with open('detections.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([class_name[2], timestamp, count, 'King Fahad Road'])

          text = f"Number of Vehicles: {count} | Traffic: {class_name[1]}"
          background_color = (136, 152, 13)

    font_scale = 0.6
    font_thickness = 1
    text_color = (255, 255, 255)

    def draw_text_with_background(frame, text, position):
        (text_w, text_h), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        cv2.rectangle(frame, (position[0], position[1] - text_h - baseline),
                      (position[0] + text_w, position[1] + baseline), background_color, -1)
        cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

    draw_text_with_background(processed_frame, text, (w - 400, 50))

    out.write(processed_frame)

Streaming output truncated to the last 5000 lines.
0: 640x384 2 cars, 1 traffic light, 1 cell phone, 16.3ms
Speed: 2.3ms preprocess, 16.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 15.1ms
Speed: 2.1ms preprocess, 15.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 1 cell phone, 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 1 cell phone, 15.1ms
Speed: 2.3ms preprocess, 15.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 1 cell phone, 15.8ms
Speed: 2.0ms preprocess, 15.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 1 cell phone, 15.2ms
Speed: 2.1ms preprocess, 15.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 cell phone, 16.2ms
Speed: 2.1ms preprocess, 16.2ms inference, 1.3ms postprocess per image at shap

In [ ]:

cap.release()
out.release()
cv2.destroyAllWindows()